In [1]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# import requests
# from config import weather_api_key


In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=1500)
lngs = np.random.uniform(-180, 180, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the lats and longs to a list
coordinates = list(lat_lngs)

In [4]:
#Create a list for holding the cities
cities=[]
#Identify the nearest city for each lat and lon combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#Print the city count tto confirm sufficient count.
#len(cities)
#print(cities)


['skjervoy', 'tanjung morawa', 'qaanaaq', 'ushuaia', 'busselton', 'bredasdorp', 'mergui', 'santa luzia', 'hermanus', 'yellowknife', 'bonavista', 'severomuysk', 'jamestown', 'kamenskoye', 'ngukurr', 'port elizabeth', 'faanui', 'rikitea', 'vila do maio', 'karaul', 'dikson', 'bosaso', 'cape town', 'nueve de julio', 'bluff', 'pombas', 'olga', 'geraldton', 'sangueya', 'chuy', 'kirakira', 'punta arenas', 'butaritari', 'dolores', 'belushya guba', 'airai', 'cairns', 'sal rei', 'san quintin', 'estelle', 'mata grande', 'kapaa', 'khatanga', 'mataura', 'saleaula', 'garowe', 'georgetown', 'barrow', 'pisco', 'fairbanks', 'lebu', 'victoria', 'merauke', 'maloshuyka', 'kiama', 'hobart', 'tsihombe', 'saint-augustin', 'muisne', 'kaitangata', 'norfolk', 'severo-kurilsk', 'taolanaro', 'palmer', 'westport', 'sigli', 'inhambane', 'vila velha', 'itarema', 'albany', 'longyearbyen', 'touros', 'jacqueville', 'barentsburg', 'mys shmidta', 'lakes entrance', 'new norfolk', 'atuona', 'guerrero negro', 'chokurdakh', 

In [5]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
#print(weather_api_key)

In [6]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_key
print(url)


http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=51c0ecd113242f8999d9c1d84d8f5b68


In [7]:
# # Create an endpoint URL for a city.
city_url = url + "&q=" + "boston"
print(city_url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=51c0ecd113242f8999d9c1d84d8f5b68&q=boston


In [8]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime


In [11]:
# CREate an empty list to hold the weather data
city_data = []
#print the beginning to the logging.
print("Beginning Data Retrieval    ")
print("----------------------------")
#create counters
record_count = 1
set_count = 1

#Loop through all the cities in our list.
for i, city in enumerate(cities):
    #Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count +=1
        record_count =1
        time.sleep(60)
        
    #Create andpoint url with each city
    #print(city)
    city_url = url + "&q=" + city.replace(" ","+")
    #With Reza, worked to find problems with API key using the print function below.)
    #print(city_url)

    #Log the url, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    #Add 1 to the record count
    record_count +=1
    
#Run and API request for each of the cities
    try:
        #Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        #parse the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_winds = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        #convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        #append the city info into city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_winds,
                          "Country": city_country,
                          "Date": city_date
        })
        
#If an error is experienced, skip the city
    except Exception as e:
        print(e)
        print("City not found. Skipping...")
        pass
    
#Indicate that Data Loading is complete
print("----------------------------")
print("Data Retrieval Complete     ")
print("----------------------------")

Beginning Data Retrieval    
----------------------------
Processing Record 1 of Set 1 | skjervoy
Processing Record 2 of Set 1 | tanjung morawa
'coord'
City not found. Skipping...
Processing Record 3 of Set 1 | qaanaaq
Processing Record 4 of Set 1 | ushuaia
Processing Record 5 of Set 1 | busselton
Processing Record 6 of Set 1 | bredasdorp
Processing Record 7 of Set 1 | mergui
'coord'
City not found. Skipping...
Processing Record 8 of Set 1 | santa luzia
Processing Record 9 of Set 1 | hermanus
Processing Record 10 of Set 1 | yellowknife
Processing Record 11 of Set 1 | bonavista
Processing Record 12 of Set 1 | severomuysk
Processing Record 13 of Set 1 | jamestown
Processing Record 14 of Set 1 | kamenskoye
'coord'
City not found. Skipping...
Processing Record 15 of Set 1 | ngukurr
'coord'
City not found. Skipping...
Processing Record 16 of Set 1 | port elizabeth
Processing Record 17 of Set 1 | faanui
Processing Record 18 of Set 1 | rikitea
Processing Record 19 of Set 1 | vila do maio
Proc

Processing Record 32 of Set 4 | bengkulu
Processing Record 33 of Set 4 | collie
Processing Record 34 of Set 4 | nhulunbuy
Processing Record 35 of Set 4 | mar del plata
Processing Record 36 of Set 4 | rawlins
Processing Record 37 of Set 4 | andenes
Processing Record 38 of Set 4 | beringovskiy
Processing Record 39 of Set 4 | puerto escondido
Processing Record 40 of Set 4 | pryazha
Processing Record 41 of Set 4 | sisimiut
Processing Record 42 of Set 4 | east london
Processing Record 43 of Set 4 | kodiak
Processing Record 44 of Set 4 | grindavik
Processing Record 45 of Set 4 | delvada
Processing Record 46 of Set 4 | vardo
Processing Record 47 of Set 4 | joutseno
Processing Record 48 of Set 4 | okhotsk
Processing Record 49 of Set 4 | siocon
Processing Record 50 of Set 4 | sterling
Processing Record 1 of Set 5 | taoudenni
Processing Record 2 of Set 5 | ho
'coord'
City not found. Skipping...
Processing Record 3 of Set 5 | codrington
Processing Record 4 of Set 5 | bloomfield
Processing Record 

Processing Record 16 of Set 8 | san cristobal
Processing Record 17 of Set 8 | hobyo
Processing Record 18 of Set 8 | aranos
Processing Record 19 of Set 8 | coquimbo
Processing Record 20 of Set 8 | rosarito
Processing Record 21 of Set 8 | kazalinsk
'coord'
City not found. Skipping...
Processing Record 22 of Set 8 | narsaq
Processing Record 23 of Set 8 | mahanje
Processing Record 24 of Set 8 | binghamton
Processing Record 25 of Set 8 | price
Processing Record 26 of Set 8 | mercedes
Processing Record 27 of Set 8 | sentyabrskiy
'coord'
City not found. Skipping...
Processing Record 28 of Set 8 | ukiah
Processing Record 29 of Set 8 | burica
'coord'
City not found. Skipping...
Processing Record 30 of Set 8 | saint george
Processing Record 31 of Set 8 | sibolga
Processing Record 32 of Set 8 | kashi
Processing Record 33 of Set 8 | hamilton
Processing Record 34 of Set 8 | ewo
Processing Record 35 of Set 8 | cabras
Processing Record 36 of Set 8 | ducheng
Processing Record 37 of Set 8 | fevralsk
'c

Processing Record 50 of Set 11 | ifanadiana
Processing Record 1 of Set 12 | meyungs
'coord'
City not found. Skipping...
Processing Record 2 of Set 12 | boueni
Processing Record 3 of Set 12 | itoman
Processing Record 4 of Set 12 | mount isa
Processing Record 5 of Set 12 | haputale
Processing Record 6 of Set 12 | zhezkazgan
Processing Record 7 of Set 12 | viedma
Processing Record 8 of Set 12 | dickinson
Processing Record 9 of Set 12 | cockburn town
Processing Record 10 of Set 12 | yumen
Processing Record 11 of Set 12 | bezdan
Processing Record 12 of Set 12 | san fernando
Processing Record 13 of Set 12 | sulangan
Processing Record 14 of Set 12 | halalo
'coord'
City not found. Skipping...
Processing Record 15 of Set 12 | svetlogorsk
Processing Record 16 of Set 12 | vrangel
Processing Record 17 of Set 12 | dhidhdhoo
Processing Record 18 of Set 12 | xunchang
Processing Record 19 of Set 12 | saint-pierre
Processing Record 20 of Set 12 | labutta
'coord'
City not found. Skipping...
Processing R

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Skjervoy,70.0311,20.9714,15.80,69,100,21.21,NO,2022-02-03 06:02:37
1,Qaanaaq,77.4840,-69.3632,-12.06,78,100,7.29,GL,2022-02-03 05:59:17
2,Ushuaia,-54.8000,-68.3000,42.46,56,75,23.02,AR,2022-02-03 06:02:53
3,Busselton,-33.6500,115.3333,94.05,32,84,8.79,AU,2022-02-03 06:03:20
4,Bredasdorp,-34.5322,20.0403,70.29,58,0,13.69,ZA,2022-02-03 06:03:20
5,Santa Luzia,-19.7697,-43.8514,68.90,96,100,3.20,BR,2022-02-03 06:03:21
6,Hermanus,-34.4187,19.2345,70.81,76,13,16.93,ZA,2022-02-03 06:03:21
7,Yellowknife,62.4560,-114.3525,-29.58,82,37,8.01,CA,2022-02-03 06:01:38
8,Bonavista,48.6499,-53.1147,28.11,71,25,21.83,CA,2022-02-03 06:03:22
9,Severomuysk,56.1481,113.4267,-11.38,89,36,5.19,RU,2022-02-03 06:03:22


In [13]:
new_column_order = ["City", "Country", "Date", "Lat","Lng","Max Temp", "Humidity", "Cloudiness", "Wind Speed"]

In [14]:
df = city_data_df[new_column_order]

In [15]:
df


,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Skjervoy,NO,2022-02-03 06:02:37,70.0311,20.9714,15.80,69,100,21.21
1,Qaanaaq,GL,2022-02-03 05:59:17,77.4840,-69.3632,-12.06,78,100,7.29
2,Ushuaia,AR,2022-02-03 06:02:53,-54.8000,-68.3000,42.46,56,75,23.02
3,Busselton,AU,2022-02-03 06:03:20,-33.6500,115.3333,94.05,32,84,8.79
4,Bredasdorp,ZA,2022-02-03 06:03:20,-34.5322,20.0403,70.29,58,0,13.69
...,...,...,...,...,...,...,...,...,...
567,Broken Hill,AU,2022-02-03 06:21:35,-31.9500,141.4333,78.19,23,0,22.39
568,Ajdabiya,LY,2022-02-03 06:21:36,30.7554,20.2263,54.28,61,22,16.11
569,Cururupu,BR,2022-02-03 06:21:36,-1.8283,-44.8683,75.52,92,99,2.89
570,Olinda,BR,2022-02-03 06:21:37,-8.0089,-34.8553,82.13,74,75,5.75


In [23]:
# Create the output file (CSV).
output_data_file = "World_Weather_Analysis/weather_data/cities.csv"
#export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")